In [52]:
import math

In [36]:
import numpy as np
from scipy.stats import norm

def approx_markov(rho, sigma_u, m, n):
    """
    Computes the Markov matrix associated with a discretized version of
    the linear Gaussian AR(1) process

        y_{t+1} = rho * y_t + u_{t+1}

    according to Tauchen's method.  Here {u_t} is an iid Gaussian
    process with zero mean.

    Parameters
    ----------
    rho : scalar(float)
        The autocorrelation coefficient
    sigma_u : scalar(float)
        The standard deviation of the random process
    m : scalar(int), optional(default=3)
        The number of standard deviations to approximate out to
    n : scalar(int), optional(default=7)
        The number of states to use in the approximation

    Returns
    -------

    x : array_like(float, ndim=1)
        The state space of the discretized process
    P : array_like(float, ndim=2)
        The Markov transition matrix where P[i, j] is the probability
        of transitioning from x[i] to x[j]

    """
    F = norm(loc=0, scale=sigma_u).cdf

    # standard deviation of y_t
    std_y = np.sqrt(sigma_u**2 / (1-rho**2))

    # top of discrete state space
    x_max = m * std_y

    # bottom of discrete state space
    x_min = - x_max

    # discretized state space
    x = np.linspace(x_min, x_max, n)

    step = (x_max - x_min) / (n - 1)
    half_step = 0.5 * step
    P = np.empty((n, n))

    for i in range(n):
        P[i, 0] = F(x[0]-rho * x[i] + half_step)
        P[i, n-1] = 1 - F(x[n-1] - rho * x[i] - half_step)
        for j in range(1, n-1):
            z = x[j] - rho * x[i]
            P[i, j] = F(z + half_step) - F(z - half_step)

    return x, P


### question (a): rho = 0.95, sigma_u = 0.007, m = 3, n = 2 

In [37]:
Z, pi = approx_markov(0.95, 0.007, 3, 2)

In [38]:
Z

array([-0.06725382,  0.06725382])

In [39]:
pi

array([[1.0000000e+00, 0.0000000e+00],
       [3.5112903e-20, 1.0000000e+00]])

### question (b): rho = 0.95, sigma_u = 0.007, m = 3, n = 7 

In [40]:
Z_2, pi_2 = approx_markov(0.95, 0.007, 3, 7)

In [41]:
Z_2

array([-0.06725382, -0.04483588, -0.02241794,  0.        ,  0.02241794,
        0.04483588,  0.06725382])

In [42]:
pi_2

array([[8.68834162e-01, 1.31158158e-01, 7.68004456e-06, 2.62012634e-14,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.73319679e-02, 8.72575630e-01, 1.00088732e-01, 3.66991038e-06,
        7.54951657e-15, 0.00000000e+00, 0.00000000e+00],
       [3.45328158e-07, 3.90841976e-02, 8.86144780e-01, 7.47689663e-02,
        1.71098555e-06, 2.10942375e-15, 0.00000000e+00],
       [5.90538639e-16, 7.78238187e-07, 5.46565099e-02, 8.90685424e-01,
        5.46565099e-02, 7.78238187e-07, 5.55111512e-16],
       [1.11029599e-28, 2.14280557e-15, 1.71098555e-06, 7.47689663e-02,
        8.86144780e-01, 3.90841976e-02, 3.45328158e-07],
       [2.14858419e-45, 6.52344432e-28, 7.58135420e-15, 3.66991038e-06,
        1.00088732e-01, 8.72575630e-01, 2.73319679e-02],
       [4.15869130e-66, 2.04854313e-44, 3.73653348e-27, 2.61545193e-14,
        7.68004456e-06, 1.31158158e-01, 8.68834162e-01]])

(i) pi_2를 transpose한 pi_2_T를 구하고 해당 matrix의 eigenvector(=mu)를 구한다

In [43]:
pi_2_T = pi_2.T

derive standard deviation and correlation coefficient of Markov process

In [48]:
eig = np.linalg.eig
mu = eig(pi_2_T)[1]
test = eig(pi_2_T)[0]

In [83]:
mu_sample = list(mu[1])
Z_sample = [math.log(Z_2[i]) for i in range(len(Z_2))]

ValueError: math domain error

In [80]:
standard_deviation = sum([(Z_sample[i]**2)*mu_sample[i] for i in range(len(Z_sample))])

In [81]:
standard_deviation

nan